In [ ]:
import time
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
def GetHistoricalData(stock, start, end, frequency):
    
    import time
    import datetime
    import requests
    import pandas as pd
    
    # Set Parameters
    start = int(time.mktime(time.strptime(start, "%m/%d/%Y")))
    end = int(time.mktime(time.strptime(end, "%m/%d/%Y")))
    assert frequency in ['daily', 'weekly', 'monthly'], 'Frequency has to be daily, weekly, or monthly'
    if frequency == 'daily':
        frequency = '1d'
    elif frequency == 'weekly':
        frequency = '1wk'
    else:
        frequency = '1m'
    
    # Get Data From Yahoo Finance
    YahooFinance = 'https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={start}&period2={end}&interval={frequency}&events=history&crumb=Vp5JJJIY4i/'
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
        'cookie': 'B=a2qoc01dom2ob&b=3&s=hv; GUC=AQEBAQFbyD5cnEIe4gR7&s=AQAAAKXb1OzH&g=W8bwqg; PRF=t%3DNFLX%252BSPY%252BRTY%253DF%252BRUTH%252B%255EIXIC%26fin-trd-cue%3D1',
        'pragma': 'no-cache',
        'referer': 'https://finance.yahoo.com/quote/{stock}/history?period1={start}&period2={end}&interval={frequency}&filter=history&frequency={frequency}'.format(stock=stock, frequency = frequency, start=start, end=end),
        'upgrade-insecure-requests': '1'
    }

    url = YahooFinance.format(stock=stock, frequency = frequency, start=start, end=end)
    response = requests.get(url, headers=headers)
    text = response.text
    lines =  text.split('\n')

    # Save Data
    title = lines[0]
    Date = []
    Open = []
    High = []
    Low = []
    Close = []
    Adj_Close = []
    Volume = []

    for line in lines[1:-1]:
        [d,o,h,l,c,ac,v] = line.split(',')
        Date.append(d)
        Open.append(float(o))
        High.append(float(h))
        Low.append(float(l))
        Close.append(float(c))
        Adj_Close.append(float(ac))
        Volume.append(float(v))
    
    # Converte Data as DataFrame
    df = pd.DataFrame({'Date': Date,
                        'Open': Open,
                        'High': High,
                        'Low': Low,
                        'Close': Close,
                        'Adj_Close': Adj_Close,
                        'Volume': Volume
                        }, columns=['Date','Open','High','Low','Close','Adj_Close','Volume'])
    df.index = pd.to_datetime(df.Date)
    df.drop("Date", inplace=True, axis=1)
    df = df.sort_index()
    
    # Calculating Daily Price Change
    df["Rate_of_Return"] = df.Close.pct_change()
    df["Price_difference"] = df.Close.diff()
    
    df["Fact_Rate_of_Return"] = df['Rate_of_Return'].shift(-1)
    
    # =1 if the stock price goes up today; =0 otherwise
    df["Up_and_Down"]= (df['Close']-df['Open'] > 0).apply(lambda x: 1 if x == True else 0) # 1:Up 0:Down
    
    # =1 if the stock price goes up tomorrow; = 0 otherwise
    df["Fact_Up_and_Down"] = df['Up_and_Down'].shift(-1)
    
    # -x: Up and down in x days ago
    for i in range(-1, -5, -1):
        df["%d_Up_and_Down" %i] = df['Up_and_Down'].shift(-1*i)

    # Adding 5-day, 15-day, 20-day and 50-day moving averages
    days = [5, 15, 20, 50]
    for ma in days:
        column_name = "MA{}".format(ma)
        df[column_name] = df.Close.rolling(ma).mean()
        
    # Calculating Daily MA curve change
    for ma in days:
        column_name = "MA{}".format(ma)
        df[column_name + "_Difference"] = df[column_name].diff()
        
    # Drop NaN values
    df = df.dropna()
    
    # Change data type(s)
    df['Fact_Up_and_Down'] = df['Fact_Up_and_Down'].astype(int)
    for i in range(-1, -5, -1):
        df["%d_Up_and_Down" % i] = df["%d_Up_and_Down" % i].astype(int)
    
    # Return the DataFrame
    return df


In [2]:
stock = 'GOOG'
start = '01/01/2000'
end = '2/19/2019'
frequency = 'daily'
df = GetHistoricalData(stock, start, end, frequency)
df

,Open,High,Low,Close,Adj_Close,Volume,Rate_of_Return,Price_difference,Fact_Rate_of_Return,Up_and_Down,...,-3_Up_and_Down,-4_Up_and_Down,MA5,MA15,MA20,MA50,MA5_Difference,MA15_Difference,MA20_Difference,MA50_Difference
Date,,,,,,,,,,,,,,,,,,,,,
2004-10-29,98.802383,99.328964,94.684174,94.704041,94.704041,42600200.0,-0.013761,-1.321403,0.028273,0,...,0,1,93.304147,79.422765,76.633323,64.010076,1.809233,1.752270,1.442120,0.897165
2004-11-01,96.149643,98.196327,95.017006,97.381630,97.381630,24608700.0,0.028273,2.677589,-0.005918,1,...,1,0,94.161571,81.435341,78.147724,64.881607,0.857423,2.012577,1.514400,0.871532
2004-11-02,98.747742,98.981224,96.045319,96.805374,96.805374,22840000.0,-0.005918,-0.576256,-0.016421,0,...,1,1,95.460124,83.338629,79.551096,65.730784,1.298553,1.903288,1.403373,0.849177
2004-11-03,98.449677,100.148628,94.758682,95.215714,95.215714,27957900.0,-0.016421,-1.589660,-0.036365,0,...,0,1,96.026441,85.020027,80.907028,66.593175,0.566316,1.681398,1.355931,0.862391
2004-11-04,93.611153,94.584816,91.082596,91.753235,91.753235,29006600.0,-0.036365,-3.462479,-0.083108,0,...,1,0,95.171999,86.434162,82.045871,67.375090,-0.854442,1.414136,1.138843,0.781914
2004-11-05,90.402023,90.560989,83.730415,84.127831,84.127831,39924100.0,-0.083108,-7.625404,0.018896,0,...,0,1,93.056757,87.270059,82.831263,67.985519,-2.115242,0.835897,0.785392,0.610430
2004-11-08,84.912727,87.153152,84.152664,85.717491,85.717491,22529100.0,0.018896,1.589660,-0.022312,1,...,0,0,90.723929,88.044688,83.757488,68.645229,-2.332828,0.774629,0.926226,0.659709
2004-11-09,86.487480,87.033928,82.101013,83.804932,83.804932,22272300.0,-0.022312,-1.912559,-0.004979,0,...,0,0,88.123841,88.732217,84.534932,69.307819,-2.600088,0.687529,0.777444,0.662591
2004-11-10,84.783562,85.702591,82.627586,83.387642,83.387642,21426400.0,-0.004979,-0.417290,0.090313,0,...,0,0,85.758226,89.638654,85.204577,69.958487,-2.365614,0.906438,0.669645,0.650668
